# CMake issues<br/> when depending on Kokkos

Damien Lebrun-Grandié and Daniel Arndt

## How to find Kokkos as an external project and load its setting?


* What are we trying to do?
* How is it done today and what are the limits of the current practice?
* What's different in the approach we suggest and why do we think it is better
* Who cares?
* What are the risks and the payoffs?


`<target>` has been created by `add_library()` or `add_executable()`

How to use say Boost.Serialization
```CMake
find_package(Boost REQUIRED serialization)
target_link_libraries(<target> Boost::serialization)
```

```C++
// Foo.cpp
#include <Kokkos_Core.hpp>

int main(int &argc, char *argv[]) {
  Kokkos::ScopeGuard(argc, argv);
  Kokkos::View<double*> v("v", 10);
}
```
```CMake
add_executable(foo.exe Foo.cpp)
# How do I link to Kokkos???
```

# Crash course: find an external project and load its settings

```CMake
find_package(<PackageName> [version] [EXACT] [QUIET] [MODULE]
             [REQUIRED] [[COMPONENTS] [components...]]
             [OPTIONAL_COMPONENTS components...]
             [NO_POLICY_SCOPE])
```
`<PackageName>_FOUND` set to indicate whether the package was found or not  
package-specific information provided through variables and **imported targets**  
`REQUIRED` option stops processing with an error message if package cannot be found  
`[version]` argument requests a version with which the pacakge found should be compatible


The command has two modes by which it searches for packages: "Module" mode and "Config" mode.  
Above signature selects Module mode and falls back to Config mode (unless `MODULE` option present)


### Module mode,
CMake searches for a file called `Find<PackageName>.cmake`.  
If found, file is read and processed by CMake.  
Responsible for finding the package, checking the version, and producing any needed messages.

### Config mode  
CMake searches for a file called `<PackageName>Config.cmake` or `<lower-case-package-name>-config.cmake`  
once found configuration file read and processed by CMake

# HOW TO BUILD KOKKOS

Kokkos can be installed via
* generated makefile
* CMake
* TriBITS as part of Trilinos

# FindKokkos

So what do we do?

Write a "find module", i.e. a `FindKokkos.cmake` file to be loaded by `find_package()`








`Kokkos_FOUND`
`Kokkos_INCLUDE_DIRS`
`Kokkos_LIBRARIES`
`Kokkos_DEFINITIONS`
`Kokkos_NVCC_WRAPPER_EXECTUABLE`
`Kokkos_LIBRARY_DIRS`
`Kokkos_VERSION`

Should **not** be used, typically cache variables for user to edit and control the behavior of find modules

`Kokkos_LIBRARY`
`Kokkos_INCLUDE_DIR`

Prevent user to be overwhelmed with settings to configure

Traditional approach is to use variables for everything, including libraries and executables

More modern approach is to behave as much like config file packages files as possible, by providing **imported target**.  
Advantage of propagating transitive usage requirements to consumers

# A sample find module for Kokkos

Try to use `pkg-config` to find the library since it is currently available for Kokkos

```CMake
find_package(PkgConfig)
pkg_check_modules(PC_Kokkos QUIET kokkos)
```
define some variables starting `PC_Kokkos_` that contain the information from the `kokkos.pc` file


use the information from `pkg-config` to provide hints to CMake about where to look
```CMake
find_path(Kokkos_INCLUDE_DIR
  NAMES Kokkos_Core.hpp
  PATHS ${PC_Kokkos_INCLUDE_DIRS}
)

find_library(Kokkos_LIBRARY
  NAMES kokkos
  PATHS ${PC_Kokkos_LIBRARY_DIRS}
)

set(Kokkos_VERSION ${PC_Kokkos_VERSION}) # not currently defined in kokkos.pc

include(FindPackageHandleStandardArgs)
find_package_handle_standard_args(Kokkos
  FOUND_VAR Kokkos_FOUND
  REQUIRED_VARS
    Kokkos_LIBRARY
    Kokkos_INCLUDE_DIR
  VERSION_VAR Kokkos_VERSION 
)
```

### Provide a way for users of the find module to link to Kokkos

Traditional variable approach looks like
```CMake
if(Kokkos_FOUND)
  set(Kokkos_LIBRARIES ${Kokkos_LIBRARY})
  set(Kokkos_INCLUDE_DIRS ${Kokkos_INCLUDE_DIR})
  set(Kokkos_DEFINITIONS ${PC_Kokkos_CFLAGS_OTHER})
endif()
```

When providing imported target, these should be namespaced (note the `Kokkos::` prefix)  
CMake will recognize that values passed to `target_link_libraries()` that contain `::` in their name are supposed to be imported targets (rather than just library names), and will produce appropriate diagnostic messages if that target does not exist 
```CMake
if(Kokkos_FOUND AND NOT TARGET Kokkos::Kokkos)
  add_library(Kokkos::Kokkos UNKNOWN IMPORTED)  
  set_target_properties(Kokkos::Kokkos PROPERTIES
    IMPORTED_LOCATION "${Kokkos_LIBRARY}"
    INTERFACE_COMPILE_OPTIONS "${PC_Kokkos_CFLAGS_OTHER}"
    INTERFACE_INCLUDE_DIRECTORIES "${Kokkos_INCLUDE_DIR}"
  )
endif()
```

```CMake
#[=======================================================================[.rst:
FindKokkos
-------

Finds the Kokkos library.

Imported Targets
^^^^^^^^^^^^^^^^

This module provides the following imported targets, if found:

``Kokkos::Kokkos``
  The Kokkos library

Result Variables
^^^^^^^^^^^^^^^^

This will define the following variables:

``Kokkos_FOUND``
  True if the system has the Kokkos library.
``Kokkos_VERSION``
  The version of the Kokkos library which was found.
``Kokkos_INCLUDE_DIRS``
  Include directories needed to use Kokkos.
``Kokkos_LIBRARIES``
  Libraries needed to link to Kokkos.

Cache Variables
^^^^^^^^^^^^^^^

The following cache variables may also be set:

``Kokkos_INCLUDE_DIR``
  The directory containing ``Kokkos_Core.hpp``.
``Kokkos_LIBRARY``
  The path to the Kokkos library.

#]=======================================================================]

find_package(PkgConfig)
pkg_check_modules(PC_Kokkos QUIET kokkos)

find_path(Kokkos_INCLUDE_DIR
  NAMES Kokkos_Core.hpp
  PATHS ${PC_Kokkos_INCLUDE_DIRS}
)
find_library(Kokkos_LIBRARY
  NAMES kokkos
  PATHS ${PC_Kokkos_LIBRARY_DIRS}
)

set(Kokkos_VERSION ${PC_Kokkos_VERSION})

include(FindPackageHandleStandardArgs)
find_package_handle_standard_args(Kokkos
  FOUND_VAR Kokkos_FOUND
  REQUIRED_VARS
    Kokkos_LIBRARY
    Kokkos_INCLUDE_DIR
  VERSION_VAR Kokkos_VERSION 
)

if(Kokkos_FOUND)
  set(Kokkos_LIBRARIES ${Kokkos_LIBRARY})
  set(Kokkos_INCLUDE_DIRS ${Kokkos_INCLUDE_DIR})
  set(Kokkos_DEFINITIONS ${PC_Kokkos_CFLAGS_OTHER})
endif()

if(Kokkos_FOUND AND NOT TARGET Kokkos::Kokkos)
  add_library(Kokkos::Kokkos UNKNOWN IMPORTED)  
  set_target_properties(Kokkos::Kokkos PROPERTIES
    IMPORTED_LOCATION "${Kokkos_LIBRARY}"
    INTERFACE_COMPILE_OPTIONS "${PC_Kokkos_CFLAGS_OTHER}"
    INTERFACE_INCLUDE_DIRECTORIES "${Kokkos_INCLUDE_DIR}"
  )
endif()

mark_as_advanced(
  Kokkos_INCLUDE_DIR
  Kokkos_LIBRARY
)
```

# NONAME

Every project that uses Kokkos write its own `FindKokkos.cmake`?

# Status quo

```diff
<prefix>
├── Makefile.kokkos
├── bin
│   └── nvcc_wrapper
├── include
│   ├── KokkosCore_config.h
│   ├── KokkosExp_MDRangePolicy.hpp
│   ├── Kokkos_AnonymousSpace.hpp
│   ├── Kokkos_Array.hpp
│   ├── Kokkos_Atomic.hpp
│   ├── Kokkos_Bitset.hpp
│   ├── Kokkos_Complex.hpp
│   ├── Kokkos_Concepts.hpp
│   ├── Kokkos_CopyViews.hpp
│   ├── Kokkos_Core.hpp
│   ├── Kokkos_Core_fwd.hpp
│   ├── Kokkos_Crs.hpp
│   ├── Kokkos_Cuda.hpp
│   ├── Kokkos_CudaSpace.hpp
│   ├── Kokkos_DualView.hpp
│   ├── Kokkos_DynRankView.hpp
│   ├── Kokkos_DynamicView.hpp
│   ├── Kokkos_ErrorReporter.hpp
│   ├── Kokkos_ExecPolicy.hpp
│   ├── Kokkos_Extents.hpp
│   ├── Kokkos_Functional.hpp
│   ├── Kokkos_Future.hpp
│   ├── Kokkos_HBWSpace.hpp
│   ├── Kokkos_HostSpace.hpp
│   ├── Kokkos_Layout.hpp
│   ├── Kokkos_Macros.hpp
│   ├── Kokkos_MasterLock.hpp
│   ├── Kokkos_MemoryPool.hpp
│   ├── Kokkos_MemoryTraits.hpp
│   ├── Kokkos_NumericTraits.hpp
│   ├── Kokkos_OffsetView.hpp
│   ├── Kokkos_OpenMP.hpp
│   ├── Kokkos_OpenMPTarget.hpp
│   ├── Kokkos_OpenMPTargetSpace.hpp
│   ├── Kokkos_Pair.hpp
│   ├── Kokkos_Parallel.hpp
│   ├── Kokkos_Parallel_Reduce.hpp
│   ├── Kokkos_PointerOwnership.hpp
│   ├── Kokkos_Profiling_ProfileSection.hpp
│   ├── Kokkos_Qthreads.hpp
│   ├── Kokkos_ROCm.hpp
│   ├── Kokkos_ROCmSpace.hpp
│   ├── Kokkos_Random.hpp
│   ├── Kokkos_ScatterView.hpp
│   ├── Kokkos_ScratchSpace.hpp
│   ├── Kokkos_Serial.hpp
│   ├── Kokkos_Sort.hpp
│   ├── Kokkos_StaticCrsGraph.hpp
│   ├── Kokkos_TaskPolicy.hpp
│   ├── Kokkos_TaskScheduler.hpp
│   ├── Kokkos_TaskScheduler_fwd.hpp
│   ├── Kokkos_Threads.hpp
│   ├── Kokkos_Timer.hpp
│   ├── Kokkos_UniqueToken.hpp
│   ├── Kokkos_UnorderedMap.hpp
│   ├── Kokkos_Vector.hpp
│   ├── Kokkos_Vectorization.hpp
│   ├── Kokkos_View.hpp
│   ├── Kokkos_WorkGraphPolicy.hpp
│   ├── Kokkos_hwloc.hpp
│   ├── OpenMP
│   │   ├── Kokkos_OpenMP_Exec.hpp
│   │   ├── Kokkos_OpenMP_Parallel.hpp
│   │   ├── Kokkos_OpenMP_Task.hpp
│   │   ├── Kokkos_OpenMP_Team.hpp
│   │   ├── Kokkos_OpenMP_ViewCopyETIAvail.hpp
│   │   ├── Kokkos_OpenMP_ViewCopyETIDecl.hpp
│   │   └── Kokkos_OpenMP_WorkGraphPolicy.hpp
│   └── impl
│       ├── KokkosExp_Host_IterateTile.hpp
│       ├── KokkosExp_ViewMapping.hpp
│       ├── Kokkos_AnalyzePolicy.hpp
│       ├── Kokkos_Atomic_Assembly.hpp
│       ├── Kokkos_Atomic_Compare_Exchange_Strong.hpp
│       ├── Kokkos_Atomic_Compare_Exchange_Weak.hpp
│       ├── Kokkos_Atomic_Decrement.hpp
│       ├── Kokkos_Atomic_Exchange.hpp
│       ├── Kokkos_Atomic_Fetch_Add.hpp
│       ├── Kokkos_Atomic_Fetch_And.hpp
│       ├── Kokkos_Atomic_Fetch_Or.hpp
│       ├── Kokkos_Atomic_Fetch_Sub.hpp
│       ├── Kokkos_Atomic_Generic.hpp
│       ├── Kokkos_Atomic_Increment.hpp
│       ├── Kokkos_Atomic_View.hpp
│       ├── Kokkos_Atomic_Windows.hpp
│       ├── Kokkos_BitOps.hpp
│       ├── Kokkos_Bitset_impl.hpp
│       ├── Kokkos_CPUDiscovery.hpp
│       ├── Kokkos_ChaseLev.hpp
│       ├── Kokkos_ClockTic.hpp
│       ├── Kokkos_ConcurrentBitset.hpp
│       ├── Kokkos_EBO.hpp
│       ├── Kokkos_Error.hpp
│       ├── Kokkos_Functional_impl.hpp
│       ├── Kokkos_FunctorAdapter.hpp
│       ├── Kokkos_FunctorAnalysis.hpp
│       ├── Kokkos_HostBarrier.hpp
│       ├── Kokkos_HostSpace_deepcopy.hpp
│       ├── Kokkos_HostThreadTeam.hpp
│       ├── Kokkos_LIFO.hpp
│       ├── Kokkos_LinkedListNode.hpp
│       ├── Kokkos_MemoryPoolAllocator.hpp
│       ├── Kokkos_Memory_Fence.hpp
│       ├── Kokkos_MultipleTaskQueue.hpp
│       ├── Kokkos_OldMacros.hpp
│       ├── Kokkos_OptionalRef.hpp
│       ├── Kokkos_PhysicalLayout.hpp
│       ├── Kokkos_Profiling_DeviceInfo.hpp
│       ├── Kokkos_Profiling_Interface.hpp
│       ├── Kokkos_Serial_Task.hpp
│       ├── Kokkos_Serial_WorkGraphPolicy.hpp
│       ├── Kokkos_SharedAlloc.hpp
│       ├── Kokkos_SimpleTaskScheduler.hpp
│       ├── Kokkos_SingleTaskQueue.hpp
│       ├── Kokkos_Spinwait.hpp
│       ├── Kokkos_StaticAssert.hpp
│       ├── Kokkos_StaticCrsGraph_factory.hpp
│       ├── Kokkos_Tags.hpp
│       ├── Kokkos_TaskBase.hpp
│       ├── Kokkos_TaskNode.hpp
│       ├── Kokkos_TaskPolicyData.hpp
│       ├── Kokkos_TaskQueue.hpp
│       ├── Kokkos_TaskQueueCommon.hpp
│       ├── Kokkos_TaskQueueMemoryManager.hpp
│       ├── Kokkos_TaskQueueMultiple.hpp
│       ├── Kokkos_TaskQueueMultiple_impl.hpp
│       ├── Kokkos_TaskQueue_impl.hpp
│       ├── Kokkos_TaskResult.hpp
│       ├── Kokkos_TaskTeamMember.hpp
│       ├── Kokkos_Timer.hpp
│       ├── Kokkos_Traits.hpp
│       ├── Kokkos_UnorderedMap_impl.hpp
│       ├── Kokkos_Utilities.hpp
│       ├── Kokkos_VLAEmulation.hpp
│       ├── Kokkos_ViewArray.hpp
│       ├── Kokkos_ViewCtor.hpp
│       ├── Kokkos_ViewFillCopyETIAvail.hpp
│       ├── Kokkos_ViewFillCopyETIDecl.hpp
│       ├── Kokkos_ViewLayoutTiled.hpp
│       ├── Kokkos_ViewMapping.hpp
│       ├── Kokkos_ViewTile.hpp
│       ├── Kokkos_ViewUniformType.hpp
│       └── Kokkos_Volatile_Load.hpp
├── kokkos_generated_settings.cmake
└── lib
    ├── cmake
    │   └── kokkos
    │       └── kokkos-config.cmake
    ├── libkokkos.a
    └── pkgconfig
        └── kokkos.pc

8 directories, 147 files
```

```cmake
find_package(Kokkos REQUIRED)
target_link_libraries(<foo> Kokkos::kokkos)
```

```cmake
find_package(PkgConfig REQUIRED)
pkg_check_modules(KOKKOS REQUIRED IMPORTED_TARGET kokkos)
#                 ^^^^^^
#                 user's choice
target_link_libraries(<foo> PkgConfig::KOKKOS)
#                                      ^^^^^^
```

* User write a `FindKokkos.cmake`

```CMake
include(FindPackageHandleStandardArgs)
find_path(KOKKOS_INCLUDE_DIR Kokkos_Core.hpp)
find_library(KOKKOS_LIBRARY NAMES kokkos)
find_package_handle_standard_args(kokkos REQUIRED_VARS
  KOKKOS_INCLUDE_DIR
  KOKKOS_LIBRARY
)

add_library(Kokkos::kokkos INTERFACE IMPORTED)
set_target_properties(Kokkos::kokkos PROPERTIES
  INTERFACE_INCLUDE_DIRECTORIES ${KOKKOS_INCLUDE_DIR}
  INTERFACE_LINK_LIBRARIES      ${KOKKOS_LIBRARY}
)
```

* What's wrong with that module?

```CMake
find_package(PkgConfig QUIET)
pkg_check_modules(PC_KOKKOS kokkos QUIET)

find_path(KOKKOS_SETTINGS_DIR kokkos_generated_settings.cmake HINTS ${PC_KOKKOS_PREFIX})

find_path(KOKKOS_INCLUDE_DIR Kokkos_Core.hpp HINTS ${PC_KOKKOS_INCLUDE_DIRS})
find_library(KOKKOS_LIBRARY NAMES kokkos kokkoscore HINTS ${PC_KOKKOS_LIBRARY_DIRS})

include(FindPackageHandleStandardArgs)
# handle the QUIETLY and REQUIRED arguments and set KOKKOS_FOUND to TRUE
# if all listed variables are TRUE
find_package_handle_standard_args(KOKKOS DEFAULT_MSG KOKKOS_SETTINGS_DIR KOKKOS_INCLUDE_DIR KOKKOS_LIBRARY)

mark_as_advanced(KOKKOS_SETTINGS_DIR KOKKOS_INCLUDE_DIR KOKKOS_LIBRARY)

if(KOKKOS_SETTINGS_DIR AND KOKKOS_INCLUDE_DIR AND KOKKOS_LIBRARY)
  include(${KOKKOS_SETTINGS_DIR}/kokkos_generated_settings.cmake)
  # https://github.com/kokkos/kokkos/issues/1838
  set(KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES_STRING)
  set(KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES)
  foreach(_f ${KOKKOS_CXX_FLAGS})
    if(NOT _f MATCHES "-I.*")
      set(KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES_STRING "${KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES_STRING} ${_f}")
      list(APPEND KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES "${_f}")
    endif()
  endforeach()
  add_library(Kokkos::kokkos UNKNOWN IMPORTED)
  set_target_properties(Kokkos::kokkos PROPERTIES
    IMPORTED_LOCATION ${KOKKOS_LIBRARY}
    INTERFACE_INCLUDE_DIRECTORIES ${KOKKOS_INCLUDE_DIR}
    INTERFACE_COMPILE_OPTIONS "${KOKKOS_CXX_FLAGS_WITHOUT_INCLUDES}"
    INTERFACE_LINK_LIBRARIES "${KOKKOS_EXTRA_LIBS}")
  # check for an empty link flags string to fix a trailing whitespace error when
  # the link flags are empty (e.g. the serial only case)
  if(KOKKOS_LINK_FLAGS)
    set_property(TARGET Kokkos::kokkos APPEND_STRING PROPERTY INTERFACE_LINK_LIBRARIES " ${KOKKOS_LINK_FLAGS}")
  endif()
endif()
```

```CMake
#[=======================================================================[.rst:

This function makes sure that kokkos was built with the requested backends
and target architectures and generates a fatal error if it was not.

kokkos_check_requested(
  [DEVICES <devices>...] # Set of backends (e.g. "OpenMP" and/or "Cuda")
  [ARCH <archs>...]      # Target architectures (e.g. "Power9" and/or "Volta70")
)
#]=======================================================================]
function(kokkos_check_requested)
  cmake_parse_arguments(_KOKKOS_REQUESTED "" "" "DEVICES;ARCH" ${ARGN})
  set(_KOKKOS_REQUESTED_ARGS)
  foreach(_X DEVICES ARCH)
    if(_KOKKOS_REQUESTED_${_X})
      list(APPEND _KOKKOS_REQUESTED_ARGS ${_X})
    endif()
  endforeach()
  set(_KOKKOS_CHECK_REQUESTED_SUCCESS TRUE)
  foreach(_X ${_KOKKOS_REQUESTED_ARGS})
    foreach(_requested ${_KOKKOS_REQUESTED_${_X}})
      foreach(_provided ${KOKKOS_${_X}})
        if(_requested STREQUAL _provided)
          string(REPLACE ";" " " ${_requested} "${_KOKKOS_REQUESTED_${_X}}")
        endif()
      endforeach()
    endforeach()
    find_package_handle_standard_args("KOKKOS_${_X}" DEFAULT_MSG ${_KOKKOS_REQUESTED_${_X}})
    if(NOT KOKKOS_${_X}_FOUND)
      set(_KOKKOS_CHECK_REQUESTED_SUCCESS FALSE)
    endif()
  endforeach()
  if(NOT _KOKKOS_CHECK_REQUESTED_SUCCESS)
    message(FATAL_ERROR "Kokkos does NOT provide all backends and/or architectures requested")
  endif()
endfunction()
```

# Summary

What do we want

1. Namespaced targets `Kokkos::`
2. Function to check at configure time that the Kokkos that was found has desirable properties
3. ???